In [2]:
import numpy as np
import pandas as pd
from scipy.stats import norm
import matplotlib.pyplot as plt
from dataclasses import dataclass
import import_ipynb
from typing import Dict, Tuple
import three_arb_check, one_iv_computation 

black_scholes = three_arb_check.black_scholes
get_spot_and_risk_free_rate = one_iv_computation.get_spot_and_risk_free_rate 
IV_surface_calls = pd.read_pickle('IV_surface_calls.pkl')
IV_surface_puts = pd.read_pickle('IV_surface_puts.pkl')

current SPY Price: $694.0700073242188
Risk-Free Rate: 0.041710000038146976 (4.171000003814697%)
Total calls: 865 (liquid: 405)

sample call options:
    strike    bid    ask  lastPrice  volume  impliedVolatility      expiry  \
1    640.0  52.62  55.41      54.37   280.0           0.772219  2026-01-12   
2    645.0  47.64  50.42      49.88   214.0           0.716311  2026-01-12   
3    650.0  42.64  45.38      44.52    23.0           0.653324  2026-01-12   
9    664.0  28.65  31.43      30.64    27.0           0.493902  2026-01-12   
10   665.0  27.64  30.43      29.05    11.0           0.481695  2026-01-12   

    dte     mid  
1     1  54.015  
2     1  49.030  
3     1  44.010  
9     1  30.040  
10    1  29.035  
successfully computed IVs, moving onto analysis...


sample comparison (c):
    strike      expiry  impliedVolatility  computed_iv   iv_diff
32   687.0  2026-01-12           0.100595     0.068897 -0.031698
33   688.0  2026-01-12           0.095346     0.109156  0.013810
34 

In [3]:
def calculate_greeks(S, K, T, r, sigma, option_type='call'):
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    
    if option_type == 'call':
        delta = norm.cdf(d1)
    else:
        delta = norm.cdf(d1) - 1
    
    gamma = norm.pdf(d1)/(S * sigma * np.sqrt(T))
    vega = S * norm.pdf(d1) * np.sqrt(T) / 100
    if option_type == 'call':
        theta = (-S * norm.pdf(d1) * sigma / (2 * np.sqrt(T)) 
                 - r * K * np.exp(-r * T) * norm.cdf(d2)) / 365
    else:
        theta = (-S * norm.pdf(d1) * sigma / (2 * np.sqrt(T)) 
                 + r * K * np.exp(-r * T) * norm.cdf(-d2)) / 365
    if option_type == 'call':
        rho = K * T * np.exp(-r * T) * norm.cdf(d2) / 100
    else:
        rho = -K * T * np.exp(-r * T) * norm.cdf(-d2) / 100
    
    return {'delta': delta,'gamma': gamma,'vega': vega,'theta': theta,'rho': rho}


In [8]:
@dataclass
class Option:
    strike: float
    dte: int  # days to expiration
    option_type: str  # 'call' or 'put'
    
    @property
    def T(self):
        return self.dte / 365.0
    
    @property
    def id(self):
        return f"{self.option_type}_{self.strike}_{self.dte}"


In [4]:
def get_mm_table(iv_grid_df, spot, T_in_years=True):
    # iv_grid_df: [moneyness, T, iv] 
    surface = (iv_grid_df.stack(dropna=True).rename("iv").reset_index())

    surface.columns = ["moneyness", "T", "iv"]

    surface["strike"] = surface["moneyness"] * spot

    if T_in_years:
        surface["dte"] = surface["T"] * 365
    else:
        surface["dte"] = surface["T"]

    surface["strike"] = surface["strike"].round(0).astype(int)
    surface["dte"] = surface["dte"].round(0).astype(int)

    mm_table = surface[["strike", "dte", "iv"]]

    return mm_table.dropna().drop_duplicates()

In [5]:
class MarketMaker:
    """
    quotes bid/ask spreads based on theo, inventory, and Greeks 
    """
    
    def __init__(self, call_surface_df, put_surface_df, spot_price, risk_free_rate=0.05):
        #dfs: [strike, dte, iv]
        self.call_surface = call_surface_df
        self.put_surface = put_surface_df
        self.spot_price = spot_price
        self.risk_free_rate = risk_free_rate
        self.inventory = {}
        
        self.base_spread = 0.02 
        self.inventory_skew_factor = 0.005 
        self.delta_limit = 100 
        self.vega_limit = 50   
        self.delta_penalty_rate = 0.01 
        self.vega_penalty_rate = 0.02 
    
    def get_iv(self, option):
        surface = self.call_surface if option.option_type == "call" else self.put_surface

        for _, row in surface.iterrows():
            if row["strike"] == option.strike and row["dte"] == option.dte:
                return float(row["iv"])

        print("no match found for this option")
        return None
    
    def get_theo(self, option):
        iv = self.get_iv(option)
        return black_scholes(self.spot_price, option.strike, option.T, 
                            self.risk_free_rate, iv, option.option_type)
    
    def get_greeks(self, option):
        iv = self.get_iv(option)
        return calculate_greeks(self.spot_price, option.strike, option.T,
                                self.risk_free_rate, iv, option.option_type
        )
    
    def get_portfolio_greeks(self):
        total_delta = 0
        total_gamma = 0
        total_vega = 0
        total_theta = 0
        
        for opt_id, position in self.inventory.items():
            ids = opt_id.split('_')
            option = Option(float(ids[1]),int(ids[2]),ids[0])
            
            greeks = self.get_greeks(option)
            total_delta += position * greeks['delta'] * 100 
            total_gamma += position * greeks['gamma'] * 100
            total_vega += position * greeks['vega'] * 100
            total_theta += position * greeks['theta'] * 100
        
        return {'delta': total_delta, 'gamma': total_gamma, 'vega': total_vega, 'theta': total_theta}
    
    def limit_penalty(self,current_exposure,trade_exposure,limit,penalty_rate,activation_fraction=0.7):

        if abs(current_exposure) <= activation_fraction * limit:
            return 0.0

        exposure_ratio = abs(current_exposure) / limit
        return penalty_rate * abs(trade_exposure) * exposure_ratio
    
    def compute_delta_penalty(self, option):
        greeks = self.get_greeks(option)
        portfolio_greeks = self.get_portfolio_greeks()
        
        current_delta = portfolio_greeks['delta']
        option_delta = greeks['delta'] * 100  # per contract
        
        return self.limit_penalty(current_exposure=current_delta,
                                trade_exposure=option_delta,
                                limit=self.delta_limit,
                                 penalty_rate=self.delta_penalty_rate)
    
    def compute_vega_penalty(self, option):
        greeks = self.get_greeks(option)
        portfolio_greeks = self.get_portfolio_greeks()
        
        current_vega = portfolio_greeks['vega']
        option_vega = greeks['vega'] * 100  # per contract
        
        return self.limit_penalty(current_exposure=current_vega,
                                trade_exposure=option_vega,
                                limit=self.delta_limit,
                                 penalty_rate=self.delta_penalty_rate)
    
    def quote(self, option):
        # quote = theo \pm skew \pm Greek penalties \pm half spread 
        
        theo = self.get_theo(option)
        half_spread = self.base_spread * abs(theo)
        position = self.inventory.get(option.id, 0)
        skew = -self.inventory_skew_factor * theo * position

        delta_penalty = self.compute_delta_penalty(option)
        vega_penalty = self.compute_vega_penalty(option)

        bid = theo - half_spread + skew - delta_penalty
        ask = theo + half_spread + skew + vega_penalty

        breakdown = {"theo": theo,"half_spread": half_spread,"inventory_skew": skew,
                    "delta_penalty": delta_penalty,"vega_penalty": vega_penalty,"position": position,
                    }

        return bid, ask, theo, breakdown
    
    def trade(self, option, side, quantity=1):
        # execute a trade. 
        self.inventory[option.id] = self.inventory.get(option.id, 0) + quantity if side == "buy" else self.inventory.get(option.id, 0) - quantity
        
    
    def get_inventory_summary(self):
        if not self.inventory:
            return pd.DataFrame()
        
        rows = []
        for option_id, position in self.inventory.items():
            if position == 0:
                continue
                
            parts = option_id.split('_')
            option = Option(float(parts[1]),int(parts[2]),parts[0])
            
            theo = self.get_theo(option)
            greeks = self.get_greeks(option)
            
            rows.append({'Option': option_id, 'Position': position, 'Strike': option.strike, 'DTE': option.dte, 'Theo': theo, 'Greeks': greeks})
        
        df = pd.DataFrame(rows)
        return df.sort_values('Option')

In [6]:
# USER: INPUT CUSTOM PARAMETERS HERE 

# Format: [Strike, DTE, Type of option, Position (sell or buy), Quantity]

option1 = [700, 8, "call", "sell", 5]
option2 = [710, 5, "put", "buy", 10]

In [ ]:
spot, rfr = get_spot_and_risk_free_rate()

call_surface_df = get_mm_table(IV_surface_calls, spot)
put_surface_df  = get_mm_table(IV_surface_puts,  spot)

mm = MarketMaker(call_surface_df, put_surface_df, spot, rfr)

# get a quote for your option 
call = Option(option1[0], option1[1], option1[2])
put = Option(option2[0], option2[1], option2[2])
bid1, ask1, theo1, components1 = mm.quote(call)
bid2, ask2, theo2, components2 = mm.quote(put)

print("Quotes (in order of user input):\n")
print(f"Bid: ${bid1:.2f}, Ask: ${ask1:.2f}, Theo: ${theo1:.2f}")
print(f"Bid: ${bid2:.2f}, Ask: ${ask2:.2f}, Theo: ${theo2:.2f}")

mm.trade(call, option1[3], option1[4]) 
mm.trade(put, option2[3], option2[4])
print(mm.get_inventory_summary())
print(mm.get_portfolio_greeks())

Quotes (in order of user input):

Bid: $1.99, Ask: $2.07, Theo: $2.03
Bid: $17.76, Ask: $18.49, Theo: $18.13
